# Отображение тем на карта

Отобразим тепловую карту тем в Санкт-Петербурге

In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('data/topics_coordinates.csv', sep='\t')
df.topic = df.topic.map(eval)

In [4]:
df.head()

,square_contain_id,keywords,topic,latitude,longitude
0,56384,"['спб', 'маникюр', 'ноготь', 'озерко', 'ресниц...","[Маникюр, Салоны красоты]",60.041502,30.325467
1,56455,"['спб', 'волос', 'ресница', 'наращивание', 'ко...",[Фитнес],60.041223,30.389278
2,56455,"['спб', 'маникюр', 'платье', 'ноготь', 'красот...","[Маникюр, Услуги красоты, Услуги фотостудий]",60.041223,30.389278
3,56455,"['спб', 'свой', 'ресторан', 'цвет', 'подарок',...","[Услуги фотостудий, Салоны красоты, Доставка т...",60.041223,30.389278
4,57010,"['акро', 'спорт', 'гимнастика', 'акробатика', ...","[Дизайн квартир/домов, Доставка товаров]",60.040760,30.315360


In [5]:
df = df.explode('topic')

In [8]:
st_districts_geojson = json.load(open('data/Regions.geojson'))

In [9]:
features = st_districts_geojson['features']

In [10]:
st_features = list(filter(lambda x: 'Санкт' in x['properties']['NAME'], features))

In [11]:
st_districts_geojson['features'] = st_features

In [12]:
json.dump(st_districts_geojson, open('st_districts.geosjon', 'w'))

In [13]:
topics = list(df.topic.value_counts().index)
np.random.shuffle(topics)

In [14]:
import os
import random

from ipywidgets import HTML, interact
from ipyleaflet import Map, GeoJSON, basemaps, Marker, Heatmap, Circle
from random import uniform
from ipywidgets.embed import embed_minimal_html


saint_petersburg_coordinates = (59.927628, 30.317613)


@interact(category=topics)
def plot_map_with_filter(category):
    m = Map(center=saint_petersburg_coordinates, zoom=10)
    
#     circle = Circle(location=(59.927628, 30.317613), radius=30, color="green", fill_color="green")
    
    heatmap = Heatmap(
        locations=[[row['latitude'], row['longitude'], 2] for row in df[(df.topic == category)].iloc],
        radius=20
    )

#     geo_json = GeoJSON(
#         data=st_districts_geojson
#     )
    
#     m.add_layer(geo_json)
    m.add_layer(heatmap)
#     m.add_layer(circle)

    return m

interactive(children=(Dropdown(description='category', options=('Дизайн квартир/домов', 'Доставка товаров', 'У…